### 📘 Talleres de Ingeniería de Datos con Pyspark en Databricks 🐍🧱

---

👨‍💻 Autor: Brayan Neciosup  
📍 Portafolio: [brayanneciosup](https://bryanneciosup626.wixsite.com/brayandataanalitics)  
🔗 LinkedIn: [linkedin.com/brayanneciosup](https://www.linkedin.com/in/brayan-rafael-neciosup-bola%C3%B1os-407a59246/)  
💻 GitHub: [github.com/BrayanR03](https://github.com/BrayanR03)  
📚 Serie: Fundamentos de Apache Spark - PySpark 
📓 Estos talleres constarán de 3 niveles (Básico-Intermedio-Avanzado)   
🔍 Abarcará temas desde Fundamentos de Data Wrangling hacia Casos de Uso Avanzado   
📝 Cada ejercicio presenta su enunciado, dataset, resultado esperado y solución.   


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder.appName("TalleresPySparkDE").getOrCreate()
### 💡 CABE RESALTAR QUE LOS DATASETS UTILIZADOS ESTAN ALMACENADOS EN UNITY CATALOG.

#### FUNDAMENTOS DE DATA WRANGLING (MANIPULACIÓN DE DATOS)

##### 🥉 NIVEL BÁSICO

In [0]:
"""
1. Detección de valores nulos en columnas principales

🗃️ Dataset: TITANIC
🗒️ Enunciado: Identifica cuántos valores faltantes hay en las columnas age, embarked y deck.
✍️ Resultado esperado: un conteo por columna con la cantidad de valores nulos.

"""
## ✔️ SOLUCIÓN
df_uno = spark.sql("SELECT * FROM workspace.exercises.titanic")
# df_uno.show(5)
cantidad_datos_nulos = df_uno.select([
  sum(when(col(c).isNull(),1).otherwise(0)).alias(c)
  for c in df_uno.columns
])
cantidad_datos_nulos.show() ## ➡️ Cantidad de datos nulos: age(177) - embarked(2) - deck(688)


In [0]:
"""
2. Eliminación de filas duplicadas

🗃️ Dataset: Diccionario
🗒️ Enunciado: Elimina las filas duplicadas y conserva solo la primera aparición de cada registro.
✍️ Resultado esperado: un DataFrame sin filas repetidas.

"""
## ✔️ Solución
dict_data = {
 "id": [1,2,2,3,4,4,5],
 "nombre": ["Ana","Luis","Luis","María","Pedro","Pedro","Sofía"],
 "edad": [23,30,30,22,40,40,29]
}
df_dos = spark.createDataFrame(data=list(zip(*dict_data.values())),schema=list(dict_data.keys()))
# df_dos.show()
print(df_dos.count()) ## Cantidad de datos originales: 7
df_dos_clean = df_dos.drop_duplicates(subset=["id","nombre","edad"])
# df_dos_clean.show()
print(df_dos_clean.count()) ## Cantidad de datos después de eliminar nulos: 5

In [0]:
"""
3. Reemplazo simple de valores faltantes

🗃️ Dataset: PENGUINS
🗒️ Enunciado: Reemplaza los valores nulos en la columna bill_length_mm con la media de esa misma columna.
✍️ Resultado esperado: columna sin valores nulos en bill_length_mm.

"""
## ✔️ Solución
df_tres = spark.sql("SELECT * FROM workspace.exercises.penguins")
# df_tres.show(5)
cantidad_nulos_df_tres = df_tres.select([
    sum(when(col(c).isNull(),1).otherwise(0)).alias(c)
    for c in df_tres.columns
    if c=='bill_length_mm'
])
# cantidad_nulos_df_tres.show() ## Cantidad de nulos: 2
media_columna_bill_length_mm = df_tres.select(round(mean(col("bill_length_mm")),2)).collect()[0][0]
media_columna_bill_length_mm ## Valor de la media: 43.92
df_tres = df_tres.fillna(value=media_columna_bill_length_mm,subset=["bill_length_mm"])

cantidad_nulos_df_tres = df_tres.select([
    sum(when(col(c).isNull(),1).otherwise(0)).alias(c)
    for c in df_tres.columns
    if c=='bill_length_mm'
])
cantidad_nulos_df_tres.show() ## Cantidad de nulos: 0

##### 🥈 NIVEL INTERMEDIO

In [0]:
import numpy as np
"""
4. Imputación condicional de valores faltantes

🗃️ Dataset: TITANIC
🗒️ Enunciado: Completa los valores faltantes de age con la edad promedio por clase (pclass).
✍️ Resultado esperado: columna age sin valores nulos, imputada según clase de pasajero.

"""
## ✔️ Solución
df_cuatro = spark.sql("SELECT * FROM workspace.exercises.titanic")
# df_cuatro.show()
cantidad_datos_nulos_df_cuatro = df_cuatro.select([
    sum(when(col(c).isNull(),1).otherwise(0)).alias(c)
    for c in df_cuatro.columns
])
# cantidad_datos_nulos_df_cuatro.select(col("age")).show() ## Cantidad de nulos: 177
promedio_edad_pclass = df_cuatro.groupBy(col("pclass")).agg(
    round(avg(col("age")),2).alias("avg_edad_pclass")
)
# promedio_edad_pclass.show()
def valor_avg_edad_pclass(pclass):
    return promedio_edad_pclass.filter(
        col("pclass")==pclass
    ).collect()[0][1]
df_cuatro_clean = df_cuatro.withColumn(
    "age",
    when(
        (col("age").isNull()==True) & (col("pclass")==1),lit(valor_avg_edad_pclass(1))
    ).when(
        (col("age").isNull()==True) & (col("pclass")==2),lit(valor_avg_edad_pclass(2))
    ).when(
        (col("age").isNull()==True) & (col("pclass")==3),lit(valor_avg_edad_pclass(3))
    ).otherwise(col("age"))
)
# df_cuatro_clean.show(5)
cantidad_datos_nulos_df_cuatro_clean = df_cuatro_clean.select([
    sum(when(col(c).isNull(),1).otherwise(0)).alias(c)
    for c in df_cuatro_clean.columns
])
# cantidad_datos_nulos_df_cuatro_clean.show() ## Cantidad de nulos: 0

In [0]:
"""
5. Detección de outliers usando IQR

🗃️ Dataset: Diccionario
🗒️ Enunciado: Identifica los valores de ventas que son outliers según el rango intercuartílico (IQR).
✍️ Resultado esperado: listado de los productos que presentan valores anómalos.

"""
## ✔️ Solución
diccionario_cinco = {
 "producto": ["A","B","C","D","E","F"],
 "ventas": [120, 130, 115, 1000, 140, 135]
}

df_cinco = spark.createDataFrame(data=list(zip(*diccionario_cinco.values())),schema=list(diccionario_cinco.keys()))
# df_cinco.show()
df_cinco_quartiles = df_cinco.agg(
    expr('percentile(ventas,array(0.25))')[0].alias("q1"),
    expr('percentile(ventas,array(0.75))')[0].alias("q3")
)
# df_cinco_quartiles.show()
q1_ventas = df_cinco_quartiles.select(col("q1")).collect()[0][0]
# q1_ventas
q3_ventas = df_cinco_quartiles.select(col("q3")).collect()[0][0]
# q3_ventas
iqr_ventas = q3_ventas - q1_ventas
lower_bound_ventas = q1_ventas - 1.5 * iqr_ventas
upper_bound_ventas = q3_ventas + 1.5 * iqr_ventas
# print(lower_bound_ventas)
# print(upper_bound_ventas)
df_cinco_outliers = df_cinco.filter(
    (col("ventas")<lower_bound_ventas) |
    (col("ventas")>upper_bound_ventas) 
)
# df_cinco_outliers.show() ## Valores outilers
# df_cinco_outliers.count() ## Cantidad de valores outilers: 1

In [0]:
"""
6. Eliminación selectiva de duplicados

🗃️ Dataset: Diamonds
🗒️ Enunciado: En el dataset de diamantes, elimina duplicados basados solo en las columnas carat y price.
✍️ Resultado esperado: DataFrame sin duplicados en esas dos columnas, pero manteniendo el resto de filas.

"""
## ✔️ Solución
df_seis = spark.sql("SELECT * FROM workspace.exercises.df_diamonds")
# df_seis.show(5)
# df_seis.count() ## Cantidad inicial de datos: 53940
df_seis_clean = df_seis.dropDuplicates(subset=["carat","price"])
# df_seis_clean.show(5)
df_seis_clean.count() ## Cantidad después de eliminar duplicados: 28988

In [0]:
"""
7. Relleno de valores faltantes con interpolación

🗃️ Dataset: Diccionario
🗒️ Enunciado: Rellena los valores nulos de la columna temperatura mediante la mediana de las temepraturas.
✍️ Resultado esperado: columna completa sin valores nulos.

"""
## ✔️ Solución
## Utilizaremos pandas para generar fechas
diccionario_siete  = {
 "fecha": ["2024-01-01","2024-01-02","2024-01-03","2024-01-04","2024-01-05","2024-01-06","2024-01-07","2024-01-08","2024-01-09","2024-01-10"],
 "temperatura": [21,22,None,24,25,None,None,28,29,30]
}
df_siete = spark.createDataFrame(data=list(zip(*diccionario_siete.values())),schema=list(diccionario_siete.keys()))
# df_siete.show()
mediana_ventas = df_siete.select(median(col("temperatura")).alias("mediana")).collect()[0][0]
# mediana_ventas
df_siete = df_siete.fillna({"temperatura":mediana_ventas})
df_siete.show()

In [0]:
"""
8. Conteo de valores faltantes combinados

🗃️ Dataset: Penguins
🗒️ Enunciado: Calcula el número de registros que tienen valores nulos simultáneamente
    en las columnas bill_length_mm y bill_depth_mm.
✍️ Resultado esperado: un número entero que indique cuántos registros cumplen esta condición.

"""
## ✔️ Solución
df_ocho = spark.sql("SELECT * FROM workspace.exercises.penguins")
# df_ocho.show(5)
df_ocho_nulos_simultaneos = df_ocho.filter(
    (col("bill_length_mm").isNull() & col("bill_depth_mm").isNull())
)
# df_ocho_nulos_simultaneos.show() ## Valores nulos simultáneos en: "bill_length_mm" y "bill_depth_mm"
df_ocho_nulos_simultaneos.count() ## Cantidad de datos que tengan valores nulos simultáneos: 2

##### 🥇 NIVEL AVANZADO

In [0]:
"""
9. Winsorización de outliers

🗃️ Dataset: Diccionario
🗒️ Enunciado: Aplica winsorización al 5% superior e inferior en la
   columna nota para reducir el impacto de valores extremos.
✍️ Resultado esperado: columna nota ajustada, sin eliminar registros.

💡 La winsorinización permite mejorar la integridad y confiabilidad de datos
    evitando valores extremos a los limites de quartil que tiene cada columna.
    Por ejemplo: valores mayores a 10, se establecen como 10 y valores
    menores a 5, se establecen como 5.

"""
## ✔️ Solución
diccionario_nueve = {
 "alumno": ["A","B","C","D","E","F","G"],
 "nota": [10, 11, 75, 100, 11, 13, 200]
}
df_nueve = spark.createDataFrame(data=list(zip(*diccionario_nueve.values())),schema=list(diccionario_nueve.keys()))
# df_nueve.show()
df_nueve_quartiles = df_nueve.agg(
    expr('percentile(nota,array(0.05))')[0].alias("q005"),
    round(expr('percentile(nota,array(0.95))')[0],2).alias("q95")
)
# df_nueve_quartiles.show()
limite_inferior_5_pct = df_nueve_quartiles.select(col("q005")).collect()[0][0]
limite_superior_95_pct = df_nueve_quartiles.select(col("q95")).collect()[0][0]
# limite_inferior_5_pct
# limite_superior_95_pct

df_nueve_final = df_nueve.withColumn(
    "nota",
    when(
        col("nota")<limite_inferior_5_pct,lit(limite_inferior_5_pct)
    ).when(
        col("nota")>limite_superior_95_pct,lit(limite_superior_95_pct)
    ).otherwise(col("nota"))
)
df_nueve_final.show()

In [0]:
"""
10. Detección de inconsistencias de tipado en columnas

🗃️ Dataset: Diccionario
🗒️ Enunciado: Identifica las filas con tipos incorrectos  y conviértelas al tipo correcto.
✍️ Resultado esperado: Un dataset con tipos de datos correcto en cada columna

"""
from pyspark.sql.types import LongType,FloatType
## ✔️ Solución
diccionario_diez = {
    "ID":[1,"2",3,"004","5"],
    "Venta":["12254",1450,1200.00,"300",120.00]
}
df_diez = spark.createDataFrame(data=list(zip(*diccionario_diez.values())),schema=list(diccionario_diez.keys()))
# df_diez.show()
df_diez_estandarizado = df_diez.withColumns({
    "ID":
    col("ID").cast(LongType()),
    "Venta":
    col("Venta").cast(FloatType())
})
df_diez.printSchema()
df_diez_estandarizado.printSchema()

In [0]:
"""
11. Identificación de duplicados aproximados (fuzzy matching)

🗃️ Dataset: Diccionario
🗒️ Enunciado: Detecta nombres de clientes que parecen duplicados
    por errores tipográficos (ejemplo: "Luis" vs "luiz").
✍️ Resultado esperado: listado de pares de valores sospechosos de ser duplicados.

"""
## ✔️ Solución
diccionario_once = {
 "cliente": ["Ana", "Ana ", "Luis", "Luz", "luiz", "Pedro", "pedroo"]
}
import difflib
lista_clientes = [i.replace(' ','') for i in diccionario_once["cliente"]]
# lista_clientes
sospechosos_duplicados = {}
for nombre in lista_clientes:
    sospechosos = difflib.get_close_matches(nombre,lista_clientes,n=3,cutoff=0.8)
    sospechosos_duplicados[nombre]=sospechosos
sospechosos_duplicados

In [0]:
"""
12. Validación y limpieza de rangos válidos

🗃️ Dataset: Titanic
🗒️ Enunciado: Valida que la columna "age" esté en un rango lógico (0 a 100 años).
               Detecta y corrige/descarta valores fuera de rango.
✍️ Resultado esperado: columna age sin valores inválidos, garantizando integridad de negocio.

"""
## ✔️ Solución
df_doce = spark.sql("SELECT * FROM workspace.exercises.titanic")
# df_doce.show(5)
df_doce_clean = df_doce.filter(
    (col("age")>0) &
    (col("age")<100) 
)
# df_doce.count() ## Cantidad de datos originales: 891
# df_doce_clean.count() ## Cantidad de datos filtrados: 714

In [0]:
"""
13. Normalización de categorías inconsistentes

🗃️ Dataset: Diccionario
🗒️ Enunciado: Detecta y unifica las categorías inconsistentes aplicando reglas 
              de limpieza (case folding, corrección de errores).
✍️ Resultado esperado: Dataset con categorías únicas y estandarizadas.

"""
## ✔️ Solución
diccionario_trece = {
    "Ciudad":["Lima","lima","LIMA","Lma",]
}
df_trece = spark.createDataFrame(data=list(zip(*diccionario_trece.values())),schema=list(diccionario_trece.keys()))
ciudades_validas = ["Lima"]
import difflib
def ciudad_estandarizada(ciudad):
    similar = difflib.get_close_matches(ciudad,ciudades_validas,n=3,cutoff=0.6)
    return similar[0] if similar else ciudad
from pyspark.sql.types import StringType
udf_ciudad = udf(ciudad_estandarizada,StringType())
df_trece = df_trece.withColumn(
    "Ciudad_Estandarizada",
    initcap(udf_ciudad(col("Ciudad")))
)
df_trece.show()


#### FEATURE ENGINEERING (INGENIERÍA DE CARACTERÍSTICAS)

##### 🥉 NIVEL BÁSICO

In [0]:
"""
1. Variables dummies simples

🗃️ Dataset: Titanic
🗒️ Enunciado: Convierte la columna sex en variables dummies
✍️ Resultado esperado: Dos columnas adicionales (sex_male, sex_female) con valores binarios 0/1.

"""
## ✔️ Solución
df_uno = spark.sql("SELECT * FROM workspace.exercises.titanic")
# df_uno.show(5)
# Esto siempre funciona y es más directo
df_with_dummies = df_uno.withColumn(
    "sex_male", 
    when(
        col("sex") == "male", 1
    ).otherwise(0)
).withColumn(
    "sex_female",
    when(
        col("sex") == "female", 1
    ).otherwise(0)
)
# df_with_dummies.show(5)
## 💡 Esta es una téncica de pre-procesamiento de datos llamada "OneHot-Encoding"
##    la cuál permite transformar datos en valores óptimos para modelos de Machine Learning (ML).

In [0]:
"""
2. Binning por intervalos fijos

🗃️ Dataset: Tips
🗒️ Enunciado: Agrupa la columna total_bill en 3 intervalos: bajo(<10), medio(>=10 y <20), alto(>=20).
✍️ Resultado esperado: Nueva columna total_bill_bin con categorías: "Bajo", "Medio", "Alto".

"""
## ✔️ Solución
df_dos = spark.sql("SELECT * FROM workspace.exercises.df_tips")
# df_dos.show(5)
df_dos = df_dos.withColumn(
    "total_bill_bin",
    when(
        col("total_bill")<10,lit("Bajo")
    ).when(
        (col("total_bill")>=10) & (col("total_bill")<20),lit("Medio")
    ).otherwise(lit("Alto"))
)
df_dos.show(5)

In [0]:
"""
3. Normalización min-max

🗃️ Dataset: Diccionario
🗒️ Enunciado: Normaliza la columna ventas entre 0 y 1 usando min-max.
✍️ Resultado esperado: Nueva columna ventas_norm con valores escalados entre 0 y 1.

"""
## ✔️ Solución
diccionario_tres = {
    "producto":["A","B","C"],
    "ventas":[100,300,500]
}
df_tres = spark.createDataFrame(data=list(zip(*diccionario_tres.values())),schema=list(diccionario_tres.keys()))
# df_tres.show()
df_tres_max_min = df_tres.agg(
    max(col("ventas")).alias("max_ventas"),
    min(col("ventas")).alias("min_ventas")
)
# df_tres_max_min.show()
df_tres = df_tres.withColumn(
    "ventas_normalizadas",
    (
        (col("ventas")-df_tres_max_min.select(col("min_ventas")).collect()[0][0])/
        (df_tres_max_min.select(col("max_ventas")).collect()[0][0] - df_tres_max_min.select(col("min_ventas")).collect()[0][0])
    )
)
df_tres.show()
# ## 💡 Esta es una téncica de pre-procesamiento de datos llamada "Normalización Min-Max"
# ##    la cuál permite establecer en un rango de 0 y 1 valores que permitan a modelos de ML
# ##    aprener los patrones de los datos, pero con variables en escalas comparables mejorando su rendimiento.


In [0]:
"""
4. Extracción de año y mes de fechas

🗃️ Dataset: Diccionario
🗒️ Enunciado: Enunciado: Extrae el año y mes de la columna fecha.
✍️ Resultado esperado: Dos nuevas columnas: año y mes.

"""
## ✔️ Solución
diccionario_cuatro = {
    "id":[1,2,3],
    "fecha":["2021-05-12","2022-01-20","2023-07-15"]
}
df_cuatro = spark.createDataFrame(data=list(zip(*diccionario_cuatro.values())),schema=list(diccionario_cuatro.keys()))
# df_cuatro.show()
df_cuatro = df_cuatro.withColumn(
    "fecha",
    to_date(col("fecha")) ## Primero convertimos a un tipo de dato fecha la columna.
)
df_cuatro = df_cuatro.withColumn(
    "año",
    year(col("fecha"))
)
# df_cuatro.show()
df_cuatro = df_cuatro.withColumn(
    "mes",
    month(col("fecha"))
)
df_cuatro.show()

In [0]:
"""
5. Longitud de cadenas de texto

🗃️ Dataset: Diccionario
🗒️ Enunciado: Crea una columna con la longitud de caracteres de cada comentario.
✍️ Resultado esperado: Columna longitud con número de caracteres por fila.

"""
## ✔️ Solución
diccionario_cinco = {
    "comentario":["Excelente servicio","Muy caro","Aceptable"]
}
df_cinco = spark.createDataFrame(data=list(zip(*diccionario_cinco.values())),schema=list(diccionario_cinco.keys()))
# df_cinco.show()
df_cinco = df_cinco.withColumn(
    "longitud_comentario",
    length(col("comentario"))
)
df_cinco.show()

##### 🥈 NIVEL INTERMEDIO

In [0]:
"""
6. Variables dummies múltiples

🗃️ Dataset: Titanic
🗒️ Enunciado: Convierte embarked en variables dummies.
✍️ Resultado esperado: Nuevas columnas (embarked_C, embarked_Q, embarked_S) con valores 0/1.

"""
## ✔️ Solución
df_seis = spark.sql("SELECT * FROM workspace.exercises.titanic")
# df_seis.show(5)
df_seis = df_seis.withColumns({
    "embarked_C":
        when(
            col("embarked")=="C",lit(1)
        ).otherwise(0),
    "embarked_Q":
        when(
            col("embarked")=="Q",lit(1)
        ).otherwise(0),
    "embarked_S":
        when(
            col("embarked")=="S",lit(1)
        ).otherwise(0)
})
display(df_seis)

In [0]:
"""
7. Binning por cuantiles

🗃️ Dataset: Diamonds
🗒️ Enunciado: Divide la columna price en 4 categorías según sus cuartiles.
✍️ Resultado esperado: Nueva columna price_bin con categorías Q1, Q2, Q3, Q4.

"""
## ✔️ Solución
df_siete = spark.sql("SELECT * FROM workspace.exercises.df_diamonds")
# df_siete.show(5)
valores_quartiles = df_siete.approxQuantile("price",[0.25,0.50,0.75],0.01)
# valores_quartiles
df_siete = df_siete.withColumn(
    "price_bin",
    when(
        col("price")<valores_quartiles[0],lit("Q1")
    ).when(
        col("price")<valores_quartiles[1],lit("Q2")
    ).when(
        col("price")<valores_quartiles[2],lit("Q3")
    ).otherwise(lit("Q4"))
)
df_siete.show()

In [0]:
"""
8. Estandarización (Z-score)

🗃️ Dataset: Diccionario
🗒️ Enunciado: Calcula el Z-score de las notas.
✍️ Resultado esperado: Nueva columna notas_zscore con valores centrados en media 0 y desviación estándar 1.

"""
## ✔️ Solución

lista_ocho = [
    12, 14, 13, 15, 16, 11, 14, 13, 15, 12,  # Notas normales
    14, 16, 13, 15, 14, 12, 13, 16, 15, 14,  # Más notas normales
    3,   # Nota muy baja (Outlier)
    19   # Nota muy lta (Outlier)
]
diccionario_ocho = {
    "Estudiante": ["Estudiante "+str(i+1) for i in range(len(lista_ocho))],
    "Nota":[i for i in lista_ocho]
}
# diccionario_ocho
df_ocho = spark.createDataFrame(data=list(zip(*diccionario_ocho.values())),schema=list(diccionario_ocho.keys()))
# df_ocho.show(5)

# 1. Calculamos el promedio y la desviación estándar de la columna "nota"
# Esto se hace sobre todo el DataFrame
promedio_nota = df_ocho.select(avg("Nota")).first()[0]
stddev_nota = df_ocho.select(stddev("Nota")).first()[0]

# 2. Realizamos el cálculo z-score
df_ocho = df_ocho.withColumn(
    "nota-zscore",
    round((col("Nota")-promedio_nota)/stddev_nota,2)
)
df_ocho.show(5)

In [0]:
"""
9. Día de la semana desde fecha

🗃️ Dataset: Diccionario
🗒️ Enunciado: Extrae el día de la semana de cada fecha.
✍️ Resultado esperado: Columna dia_semana con valores tipo: "Viernes", "Sábado", "Domingo".

"""
## ✔️ Solución
diccionario_nueve = {
    "id":[1,2,3],
    "fecha":["2023-05-12","2023-05-13","2023-05-14"]
}
df_nueve = spark.createDataFrame(data=list(zip(*diccionario_nueve.values())),schema=list(diccionario_nueve.keys()))
# df_nueve.show(5)
df_nueve = df_nueve.withColumn(
    "fecha",
    to_date(col("fecha"))
)
df_nueve = df_nueve.withColumn(
    "dia_semana",
    date_format(col("fecha"),"EEEE")
)
df_nueve.show()
## 💡 Es importante conocer los diversos formatos de fechas.

In [0]:
"""
10. Conteo de palabras en texto

🗃️ Dataset: Diccionario
🗒️ Enunciado: Crea una columna con el número de palabras por comentario.
✍️ Resultado esperado: Columna num_palabras con valores 4, 4, 3.

"""
## ✔️ Solución

diccionario_diez = {
    "comentario":["Me gusta el servicio","Precio alto pero bueno","No lo recomiendo"]
}
df_diez = spark.createDataFrame(data=list(zip(*diccionario_diez.values())),schema=list(diccionario_diez.keys()))
df_diez = df_diez.withColumn(
    "cantidad_palabras",
    size(split(col("comentario")," "))
)
df_diez.show()

##### 🥇 NIVEL AVANZADO

In [0]:
from pyspark.sql.types import StringType
"""
11. Variables categóricas cruzadas

🗃️ Dataset: Titanic
🗒️ Enunciado: Crea una nueva variable categórica que combine pclass y sex (ejemplo: "1_female", "3_male").
✍️ Resultado esperado: Columna clase_sexo con las combinaciones únicas.

"""
## ✔️ Solución
df_once = spark.sql("SELECT * FROM workspace.exercises.titanic")
# df_once.show(5)
df_once = df_once.withColumn(
    "clase_sexo",
    concat_ws("-",col("pclass"),col("sex"))
)
def funcion_dummies_clase_sexo(df,columna):
    df_temp = df.select(col(columna))
    df_temp = df_temp.dropDuplicates(subset=[columna]).orderBy(col(columna))
    for i in df_temp.collect():
        df = df.withColumn(
            i[0],
            when(
                col("clase_sexo")==i[0],lit(1)
            ).otherwise(lit(0))
        )
    return df
df_once_final = funcion_dummies_clase_sexo(df_once,"clase_sexo")
display(df_once_final)

In [0]:
"""
12. Binning desigual basado en reglas

🗃️ Dataset: Tips
🗒️ Enunciado: Clasifica tip en categorías: "Bajo" (<2), "Medio" (2 - 5), "Alto" (>5).
✍️ Resultado esperado: Nueva columna tip_categoria con esas 3 categorías.

"""
## ✔️ Solución
df_doce = spark.sql("SELECT * FROM workspace.exercises.df_tips")
# df_doce.show()
df_doce = df_doce.withColumn(
    "tip_categoria",
    when(
        col("tip")<2,lit("Bajo")
    ).when(
        (col("tip")>=2) & (col("tip")<=5),lit("Medio")
    ).otherwise(lit("Alto"))
)
df_doce.show(5)

In [0]:
"""
13. Normalización robusta

🗃️ Dataset: Diccionario
🗒️ Enunciado: Aplica una normalización robusta (usando mediana e IQR).
✍️ Resultado esperado: Columna ventas_robust que reduzca la influencia de outliers.

"""
## ✔️ Solución
diccionario_trece = {
    "producto":["A","B","C","D"],
    "ventas":[10,200,1000,10000]
}
df_trece = spark.createDataFrame(data=list(zip(*diccionario_trece.values())),schema=list(diccionario_trece.keys()))
# df_trece.show()

##  ✅ Hallamos los Quartiles (columna ventas)
df_trece_quartiles = df_trece.agg(
    expr('percentile(ventas,array(0.25))')[0].alias("q1"),
    expr('percentile(ventas,array(0.75))')[0].alias("q3")
)
# df_trece_quartiles.show()

## ✅ Hallamos IQR (Rango Interquartil - columnas ventas)
iqr_ventas = df_trece_quartiles.select(col("q3")).collect()[0][0] - df_trece_quartiles.select(col("q1")).collect()[0][0]
# iqr_ventas

## ✅ Hallamos mediana (columna ventas)
mediana_ventas = df_trece.select(median(col("ventas"))).collect()[0][0]
# mediana_ventas

## ✅ Calculamos Normalización robusta
df_trece = df_trece.withColumn(
    "ventas_robus",
    round((col("ventas") - mediana_ventas)/iqr_ventas,2)
)
df_trece.show()

In [0]:
from pyspark.sql.types import TimestampType
"""
14. Extracción de partes avanzadas de fecha

🗃️ Dataset: Diccionario
🗒️ Enunciado: Extrae la hora, el minuto y el nombre del día de la semana.
✍️ Resultado esperado: Tres nuevas columnas: hora, minuto, dia_semana.

"""
## ✔️ Solución
diccionario_catorce = {
    "id":[1,2,3],
    "fecha":["2021-05-12 14:35:00","2021-05-12 20:10:00","2021-05-13 08:45:00"]
}
df_catorce = spark.createDataFrame(data=list(zip(*diccionario_catorce.values())),schema=list(diccionario_catorce.keys()))
# df_catorce.show()
# df_catorce.printSchema()
df_catorce = df_catorce.withColumn(
    "fecha",
    col("fecha").cast(TimestampType())
)
# df_catorce.show()
# df_catorce.printSchema()
df_catorce = df_catorce.withColumns({
    "hora":
    hour(col("fecha")),
    "minuto":
        minute(col("fecha")),
    "dia_semana":
    date_format(col("fecha"),"EEEE")
})
df_catorce.show()

In [0]:
from pyspark.sql.types import IntegerType
"""
15. Extracción de features de texto (tokens únicos)

🗃️ Dataset: Diccionario
🗒️ Enunciado: Crea una columna con el número de palabras únicas en cada comentario.
✍️ Resultado esperado: Columna palabras_unicas con valores (3, 2, 4).

"""
## ✔️ Solución
diccionario_quince = {
    "comentario":["Muy buen buen servicio","Servicio aceptable aceptable","No me gustó el servicio"]
}
df_quince = spark.createDataFrame(data=list(zip(*diccionario_quince.values())),schema=list(diccionario_quince.keys()))
df_quince = df_quince.select(
    "*",
    size(array_distinct(split(col("comentario")," ",0))).alias("palabras_unicas")
)
display(df_quince)